In [1]:
from artists_88rising import artists
import requests
from config import client_id, client_secret 
import base64
import datetime
from urllib.parse import urlencode

ImportError: cannot import name 'client_id' from 'config' (/Users/og/opt/anaconda3/lib/python3.9/site-packages/config/__init__.py)

In [ ]:
client_creds = f"{client_id}:{client_secret}"
client_creds_b64 = base64.b64encode(client_creds.encode())
base64.b64decode(client_creds_b64)

In [ ]:
# do a looklup for a token and future reqeust 

token_url = 'https://accounts.spotify.com/api/token'
method = 'POST'
token_data = {
    'grant_type': "client_credentials"
}
token_headers = {
    "Authorization": f"Basic {client_creds_b64.decode()}" #<base64 encoded client_id:client_secret>
}

In [ ]:
r = requests.post(token_url, data=token_data, headers=token_headers)
print(r.json())
valid_request = r.status_code in range(200,299)
token_response_data = r.json()

In [ ]:
access_token = token_response_data['access_token']

In [ ]:
#Spotify Search and extract their Spotify ID
def searchArtist(name) :
    headers = {
        'Authorization': f'Bearer {access_token}'
    }
    endpoint = 'https://api.spotify.com/v1/search'
    data = urlencode({'q': name, 'type': 'artist'})
    top = urlencode({'offset': '0', 'limit': '20'})

    query_url = f'{endpoint}?{data}&{top}'

    r = requests.get(query_url, headers=headers) 
    artist_data=r.json()
    id = artist_data['artists']['items'][0]['id']
    return id

In [ ]:
# List of artists in 88 Rising
artists

In [ ]:
# Create a dictionary to extract their Spotify ID
artists_dict = {}
for a in artists:
    a_id = searchArtist(a)
    artists_dict[a] = a_id

In [ ]:
artists_dict

In [ ]:
# Now that we have the ID's we can pull all the albums from each artist
headers = {
        'Authorization': f'Bearer {access_token}'
    }
album_url = f'https://api.spotify.com/v1/artists/{artists_dict["Rich Brian"]}/albums'
r = requests.get(album_url, headers=headers) 
album_data = r.json()



In [ ]:
# Example
rich_brian_album = album_data['items'][1]['name']
rich_brian_album_id = album_data['items'][1]['id']
print(rich_brian_album, rich_brian_album_id)

In [ ]:
# Get Tracks of each album
tracks_url = f'https://api.spotify.com/v1/albums/{rich_brian_album_id}/tracks'
t = requests.get(tracks_url, headers=headers) 
track_data = t.json()

In [ ]:
track_name = track_data['items'][1]['name']
track_id = track_data['items'][1]['id']
print(track_name, track_id)

In [ ]:
# Get Audio Features
audio_url = f'https://api.spotify.com/v1/audio-features/{track_id}'
audio = requests.get(audio_url, headers= headers)
audio_data = audio.json()

In [ ]:
audio_data